# Обучение МЛ моделей 

- Импорт библиотек

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm as tqdm
import sklearn
sklearn.set_config(transform_output='pandas')
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler, OrdinalEncoder, TargetEncoder,  LabelEncoder
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression , Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, VotingClassifier, VotingRegressor, BaggingClassifier, BaggingRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score
import optuna
from sklearn.svm import SVR
import time
import joblib

/Users/verakabanova/Desktop/Classic ML.py/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Считываем данные

In [3]:
path ='/Users/verakabanova/Desktop/Classic ML.py/cleaned_real_estate.csv'

In [4]:
df = pd.read_csv(path)

In [5]:
df.head()

,ad_id,property_type,metro,address,total_area,parking,price,renovation,balcony,windows,bathroom,children_pets,amenities,ceiling_height,elevator,numbere_of_rooms,Time_metro
0,273973191,Квартира,Смоленская,"Москва, Новинский бульвар, 18С1",120.0,открытая,130000.0,Евроремонт,Балкон (1),На улицу,Совмещенный (1),Можно с животными,"Мебель на кухне, Ванна, Стиральная машина, Кон...",3.0,Пасс (1),3,9.0
1,273999490,Квартира,Смоленская,"Москва, Новинский бульвар, 7",80.0,наземная,100000.0,Евроремонт,Балкон (3),На улицу и двор,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Душевая ка...",2.4,Пасс (1),3,2.0
2,274995950,Квартира,Арбатская,"Москва, улица Новый Арбат, 15",30.0,наземная,120000.0,Евроремонт,Балкон (1),На улицу,Совмещенный (1),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",2.4,Пасс (1),2,10.0
3,271265359,Квартира,Смоленская,"Москва, улица Арбат, 45/24",60.0,наземная,90000.0,Евроремонт,Балкон (1),На улицу и двор,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.2,Пасс (1),2,5.0
4,273779074,Квартира,Смоленская,"Москва, Большой Николопесковский переулок, 3",60.0,наземная,120000.0,Косметический,Балкон (1),На улицу,Совмещенный (1),"Можно с детьми, Можно с животными","Мебель в комнатах, Ванна, Стиральная машина, К...",3.6,Пасс (1),2,7.0


# Preprocessing (Подготовка данных)

In [6]:
df.isnull().sum()

ad_id               0
property_type       0
metro               0
address             0
total_area          0
parking             0
price               0
renovation          0
balcony             0
windows             0
bathroom            0
children_pets       0
amenities           0
ceiling_height      0
elevator            0
numbere_of_rooms    0
Time_metro          0
dtype: int64

- заменяем текст в колонке на его длинну

In [7]:
df['amenities'] = df['amenities'].apply(lambda x: len(str(x).split(',')))


- Кодировка категориальных переменных с присвоением порядка

In [8]:
for column in ['renovation', 'balcony', 'windows', 'parking', 'children_pets', 'bathroom']:
    if column in df.columns:
        df[f'{column}_encoded'] = df[column].astype('category').cat.codes
        print(f'{column} закодированно')

renovation закодированно
balcony закодированно
windows закодированно
parking закодированно
children_pets закодированно
bathroom закодированно


In [9]:

windows_mapping = {
    'Во двор': 0,
    'На улицу': 1, 
    'На улицу и двор': 2
}

# Применяем только для окон
df['windows_encoded'] = df['windows'].map(windows_mapping)

print("✅ Windows закодированы!")
print(df[['windows', 'windows_encoded']].head())

✅ Windows закодированы!
           windows  windows_encoded
0         На улицу                1
1  На улицу и двор                2
2         На улицу                1
3  На улицу и двор                2
4         На улицу                1


In [10]:
# Пассажирские лифты
df['pass_elevators'] = df['elevator'].str.extract('Пасс \((\d+)\)').fillna(0).astype(int)

# Грузовые лифты  
df['cargo_elevators'] = df['elevator'].str.extract('Груз \((\d+)\)').fillna(0).astype(int)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:5: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:5: SyntaxWarning: invalid escape sequence '\('
/var/folders/pm/nngz2k053tzftqqv04ft8wt40000gn/T/ipykernel_4591/3038078685.py:2: SyntaxWarning: invalid escape sequence '\('
  df['pass_elevators'] = df['elevator'].str.extract('Пасс \((\d+)\)').fillna(0).astype(int)
/var/folders/pm/nngz2k053tzftqqv04ft8wt40000gn/T/ipykernel_4591/3038078685.py:5: SyntaxWarning: invalid escape sequence '\('
  df['cargo_elevators'] = df['elevator'].str.extract('Груз \((\d+)\)').fillna(0).astype(int)


In [11]:
dummi = pd.get_dummies(df['property_type'], prefix = 'property')
df = pd.concat([df, dummi], axis=1)
df = df.drop('property_type', axis=1 )

In [12]:
metro_target = df.groupby('metro')['price'].mean()
df['metro_encoder'] = df['metro'].map(metro_target)


In [13]:
address_encod = df['address'].value_counts(normalize=True)
df['address_encod']  = df['address'].map(address_encod)

- удаляем закодированые колонки 

In [14]:
df = df.drop(['parking', "renovation", 'balcony', 'windows', 'bathroom', 'children_pets', 'elevator'], axis = 1)

In [15]:
df = df.drop(['metro', 'address', 'ad_id'], axis=1)

In [16]:
df.head()

,total_area,price,amenities,ceiling_height,numbere_of_rooms,Time_metro,renovation_encoded,balcony_encoded,windows_encoded,parking_encoded,children_pets_encoded,bathroom_encoded,pass_elevators,cargo_elevators,property_Квартира,metro_encoder,address_encod
0,120.0,130000.0,8,3.0,3,9.0,2,0,1,3,2,4,1,0,True,103000.000000,0.000049
1,80.0,100000.0,8,2.4,3,2.0,2,8,2,2,0,4,1,0,True,103000.000000,0.000049
2,30.0,120000.0,11,2.4,2,10.0,2,0,1,2,1,4,1,0,True,107142.857143,0.000098
3,60.0,90000.0,11,3.2,2,5.0,2,0,2,2,0,4,1,0,True,103000.000000,0.000049
4,60.0,120000.0,6,3.6,2,7.0,3,0,1,2,1,4,1,0,True,103000.000000,0.000098


In [17]:
df.columns

Index(['total_area', 'price', 'amenities', 'ceiling_height',
       'numbere_of_rooms', 'Time_metro', 'renovation_encoded',
       'balcony_encoded', 'windows_encoded', 'parking_encoded',
       'children_pets_encoded', 'bathroom_encoded', 'pass_elevators',
       'cargo_elevators', 'property_Квартира', 'metro_encoder',
       'address_encod'],
      dtype='object')

- Подготовка для обучения . Разделяем данные и исключаем целевую переменную

In [18]:
X = df.drop('price', axis=1)
y = df['price']

In [19]:
X.head()

,total_area,amenities,ceiling_height,numbere_of_rooms,Time_metro,renovation_encoded,balcony_encoded,windows_encoded,parking_encoded,children_pets_encoded,bathroom_encoded,pass_elevators,cargo_elevators,property_Квартира,metro_encoder,address_encod
0,120.0,8,3.0,3,9.0,2,0,1,3,2,4,1,0,True,103000.000000,0.000049
1,80.0,8,2.4,3,2.0,2,8,2,2,0,4,1,0,True,103000.000000,0.000049
2,30.0,11,2.4,2,10.0,2,0,1,2,1,4,1,0,True,107142.857143,0.000098
3,60.0,11,3.2,2,5.0,2,0,2,2,0,4,1,0,True,103000.000000,0.000049
4,60.0,6,3.6,2,7.0,3,0,1,2,1,4,1,0,True,103000.000000,0.000098


- Нормализуем данные 

In [20]:
numerical_cols = X.select_dtypes(include=[np.number]).columns

scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

- Разделяем на обучающую и валидационную выборку

In [21]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание ColumnTransformer / Pipeline

- если новые данные будут иметь пропуски они автоматически заполнятся выбраной стратегией 

In [22]:
numeric_features = X_train.select_dtypes(include=[np.number]).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

In [23]:
preprocessor = ColumnTransformer(
    transformers= [
        ('numeric', Pipeline(steps=[
             ('imputer', SimpleImputer(strategy='median')),
             ('scaler', StandardScaler())
        ]), numeric_features),

        ('categorical', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
            ('onehot', OneHotEncoder(
                handle_unknown="ignore", sparse_output=False, drop="first"
            ))
        ]), categorical_features)
       
        
    ], 
    remainder= 'passthrough',
    n_jobs=-1,
    verbose_feature_names_out= False,
    
)

In [24]:
full_pipeline = Pipeline([
    ('preprocessor', preprocessor) , 
('model', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])

full_pipeline.fit(X_train, y_train)

y_pred = full_pipeline.predict(X_valid)

mae = mean_absolute_error(y_valid, y_pred)
r2 = r2_score(y_valid, y_pred)

print(f'MAE = ${mae:,.0f}, r2 = {r2:.3f}')

MAE = $7,424, r2 = 0.789


- Сохранение пайплайна

In [30]:
joblib.dump(full_pipeline, "real_estate_pipeline_v1.pkl")
print('Пайплайн сохранен!')

Пайплайн сохранен!


- Тестирование других моделей 

In [31]:
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42), 
    'Ridge' : Ridge(alpha=1.0),
    'Gradient Boosting' : GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# Сравнение моделей 
for name, model in models.items():
    test_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

# Кросс-валидация 
cv_scores = cross_val_score(test_pipeline, X_train, y_train, cv=3, scoring='r2', n_jobs=-1)

print(f" {name:20} | R2: {cv_scores.mean():.3f} -+ {cv_scores.std():.3f}")

 Gradient Boosting    | R2: 0.767 -+ 0.006


- Анализ важности признаков

In [32]:
trained_model = full_pipeline.named_steps['model']

feature_names = full_pipeline.named_steps['preprocessor'].get_feature_names_out()

feature_importance = pd.DataFrame({'feature': feature_names,
                                   'importance': trained_model.feature_importances_
                                   }).sort_values('importance', ascending=False)

print(feature_importance.head())

               feature  importance
0           total_area    0.477013
13       metro_encoder    0.233631
5   renovation_encoded    0.046337
3     numbere_of_rooms    0.036895
1            amenities    0.036458


- Настройка гиперпараметров 

In [33]:
param_grid = {
    'model__n_estimators':[100, 200],
    'model__max_depth': [15, 20, None],
    'model__min_samples_split': [2, 5]
}
grid_search = GridSearchCV(full_pipeline, param_grid, cv=3, scoring='r2', n_jobs=-1, 
                           verbose=1)
grid_search.fit(X_train, y_train)

print(f' Лучшие параметры : {grid_search.best_params_}')
print(f'Лучший R2 : {grid_search.best_score_:.3f}')

Fitting 3 folds for each of 12 candidates, totalling 36 fits
 Лучшие параметры : {'model__max_depth': 15, 'model__min_samples_split': 5, 'model__n_estimators': 200}
Лучший R2 : 0.773


# Обучение финальной модели с лучшими параметрами 

In [35]:
final_pipeline = Pipeline([
    ('preprocessor', preprocessor), 
    ('model', RandomForestRegressor(
        n_estimators = 200,
        max_depth = 15,
        min_samples_split= 5, 
        min_samples_leaf= 2,
        n_jobs =-1 ,
        random_state = 42
    ))
])

final_pipeline.fit(X_train, y_train)
print('Финальная модель обучена с лучшшеми параметрами!')

Финальная модель обучена с лучшшеми параметрами!


- Оценка на валидационной выборке

In [36]:
y_pred_final = final_pipeline.predict(X_valid)

mae_final = mean_absolute_error(y_valid, y_pred_final)
r2_final = r2_score(y_valid, y_pred_final)

print(f' MAE : ${mae_final:,.0f}')
print(f'R2 : {r2_final:.3f}')
print(f'Средняя цена : ${y_valid.mean():,.0f}')
print(f' Точность : {(1 - mae_final/y_valid.mean())* 100:.1f}%')


# Сравнение с предыдущей моделью 
improvement = r2_final - 0.789
print(f' Изменение R2 : {improvement:+.3f}')

 MAE : $7,412
R2 : 0.790
Средняя цена : $51,397
 Точность : 85.6%
 Изменение R2 : +0.001


# Сохраняем финальную модель 

In [38]:
joblib.dump(final_pipeline, 'final_real_estate_pipeline.pkl')
print('ФИНАЛЬНАЯ МОДЕЛЬ СОХРАНЕНА!')

ФИНАЛЬНАЯ МОДЕЛЬ СОХРАНЕНА!
